# 8.1 Hierarchical Indexing

In [0]:
import pandas as pd
import numpy as np

In [2]:
data = pd.Series(np.random.randn(9),
                 index = [['a', 'a', 'a', 'b', 'b', 'c', 'c',
                           'd', 'd'],
                          [1, 2, 3, 1, 3, 1, 2, 2, 3]])

data

a  1   -1.538393
   2   -1.495889
   3   -2.063205
b  1    0.449050
   3   -1.316248
c  1   -0.018875
   2   -0.908130
d  2    0.417537
   3   -1.135896
dtype: float64

In [3]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [4]:
data['b']

1    0.449050
3   -1.316248
dtype: float64

In [5]:
data['b' : 'c']

b  1    0.449050
   3   -1.316248
c  1   -0.018875
   2   -0.908130
dtype: float64

In [6]:
data.loc[['b', 'd']]

b  1    0.449050
   3   -1.316248
d  2    0.417537
   3   -1.135896
dtype: float64

In [7]:
data.loc[:, 2]

a   -1.495889
c   -0.908130
d    0.417537
dtype: float64

In [8]:
data.unstack()

,1,2,3
a,-1.538393,-1.495889,-2.063205
b,0.449050,NaN,-1.316248
c,-0.018875,-0.908130,NaN
d,NaN,0.417537,-1.135896


In [9]:
data.unstack().stack()

a  1   -1.538393
   2   -1.495889
   3   -2.063205
b  1    0.449050
   3   -1.316248
c  1   -0.018875
   2   -0.908130
d  2    0.417537
   3   -1.135896
dtype: float64

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index = [['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns = [['Ohio', 'Ohio', 'Colorado'],
                                ['Green', 'Red', 'Green']])

frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [0]:
frame.index.names = ['key1', 'key2']

In [0]:
frame.columns.names = ['state', 'color']

In [13]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [14]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [15]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'],
                        ['Green', 'Red', 'Green']],
                       names = ['state', 'color'])

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           labels=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

## Reordering and Sorting Levels

In [16]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [17]:
frame.sort_index(level = 1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [18]:
frame.swaplevel(0, 1).sort_index(level = 0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## Summary Statistics by Level

In [19]:
frame.sum(level = 'key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [20]:
frame.sum(level = 'color', axis = 1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## Indexing with a DataFrame's columns

In [21]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})

frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [22]:
frame2 = frame.set_index(['c', 'd'])

frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [23]:
frame.set_index(['c', 'd'], drop = False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [24]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# 8.2 Combining and Merging Datasets

## Database - Style DataFrame Joins

In [25]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [26]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

df2

,data2,key
0,0,a
1,1,b
2,2,d


In [27]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [28]:
pd.merge(df1, df2, on = 'key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [29]:
df3 = df1.copy()

df3.columns = ['data1', 'lkey']

df3

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [30]:
df4 = df2.copy()

df4.columns = ['data2', 'rkey']

df4

,data2,rkey
0,0,a
1,1,b
2,2,d


In [31]:
pd.merge(df3, df4, left_on = 'lkey', right_on = 'rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [32]:
pd.merge(df1, df2, how = 'outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [33]:
pd.merge(df1, df2, how = 'inner')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [34]:
pd.merge(df1, df2, how = 'left')

,data1,key,data2
0,0,b,1.0
1,1,b,1.0
2,2,a,0.0
3,3,c,NaN
4,4,a,0.0
5,5,a,0.0
6,6,b,1.0


In [35]:
pd.merge(df1, df2, how = 'right')

,data1,key,data2
0,0.0,b,1
1,1.0,b,1
2,6.0,b,1
3,2.0,a,0
4,4.0,a,0
5,5.0,a,0
6,NaN,d,2


In [36]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})

df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [37]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})

df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [38]:
pd.merge(df1, df2, on = 'key', how = 'left')

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


In [39]:
pd.merge(df1, df2, how = 'inner')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [40]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})

left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [41]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [42]:
pd.merge(left, right, on = ['key1', 'key2'], how = 'outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [43]:
pd.merge(left, right, on = 'key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [44]:
pd.merge(left, right, on = 'key1', suffixes = ('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


## Examples explaining merge function arguments

In [45]:
df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [1, 2, 3, 5]})

df1

,lkey,value
0,foo,1
1,bar,2
2,baz,3
3,foo,5


In [46]:
df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [5, 6, 7, 8]})

df2

,rkey,value
0,foo,5
1,bar,6
2,baz,7
3,foo,8


In [47]:
df1.merge(df2, left_on = 'lkey', right_on = 'rkey')

,lkey,value_x,rkey,value_y
0,foo,1,foo,5
1,foo,1,foo,8
2,foo,5,foo,5
3,foo,5,foo,8
4,bar,2,bar,6
5,baz,3,baz,7


In [48]:
df1.merge(df2, left_on = 'lkey', right_on = 'rkey',
          suffixes = ('_left', '_right'))

,lkey,value_left,rkey,value_right
0,foo,1,foo,5
1,foo,1,foo,8
2,foo,5,foo,5
3,foo,5,foo,8
4,bar,2,bar,6
5,baz,3,baz,7


In [0]:
# df1.merge(df2, left_on = 'lkey', right_on = 'rkey',
#             suffixes = (False, False))

# ValueError: columns overlap but no suffix specified

In [50]:
# https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/

# user_usage = pd.read_csv("https://github.com/shanealynn/Pandas-Merge-Tutorial/blob/master/user_usage.csv")

# user_device = pd.read_csv("https://github.com/shanealynn/Pandas-Merge-Tutorial/blob/master/user_device.csv")

# android_devices = pd.read_csv("https://github.com/shanealynn/Pandas-Merge-Tutorial/blob/master/android_devices.csv")

from google.colab import files

uploaded = files.upload()

Saving user_device.csv to user_device.csv


In [51]:
uploaded = files.upload()

Saving user_usage.csv to user_usage.csv


In [52]:
uploaded = files.upload()

Saving android_devices.csv to android_devices.csv


In [0]:
user_usage = pd.read_csv("user_usage.csv")

user_device = pd.read_csv("user_device.csv")

android_devices = pd.read_csv("android_devices.csv")

In [54]:
user_usage.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


In [55]:
user_device.head()

,use_id,user_id,platform,platform_version,device,use_type_id
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3
2,22784,28473,android,5.1,SM-G903F,1
3,22785,15200,ios,10.2,"iPhone7,2",3
4,22786,28239,android,6.0,ONE E1003,1


In [56]:
android_devices.head(10)

,Retail Branding,Marketing Name,Device,Model
0,NaN,NaN,AD681H,Smartfren Andromax AD681H
1,NaN,NaN,FJL21,FJL21
2,NaN,NaN,T31,Panasonic T31
3,NaN,NaN,hws7721g,MediaPad 7 Youth 2
4,3Q,OC1020A,OC1020A,OC1020A
5,7Eleven,IN265,IN265,IN265
6,A.O.I. ELECTRONICS FACTORY,A.O.I.,TR10CS1_11,TR10CS1
7,AG Mobile,AG BOOST 2,BOOST2,E4010
8,AG Mobile,AG Flair,AG_Flair,Flair
9,AG Mobile,AG Go Tab Access 2,AG_Go_Tab_Access_2,AG_Go_Tab_Access_2


In [57]:
result = pd.merge(user_usage,
                  user_device[['use_id', 'platform', 'device']],
                  on = 'use_id')

result.head()

## We start with the user_usage DF (the 'left' dataset in the merge)
## We are joining data from the user_device DF (the 'right' dataset)
## Common column b/w right and left DFs is 'use_id'
## We only want 3 columns (use_id, platform and device) from the
## user_device dataset

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


In [58]:
print("user_usage dimensions: {}".format(user_usage.shape))
print("user_device dimensions: {}".format(user_device[['use_id', 'platform', 'device']].shape))
print("result dimensions: {}".format(result.shape))

user_usage dimensions: (240, 4)
user_device dimensions: (272, 3)
result dimensions: (159, 6)


In [59]:
## By default, Pandas merge operation acts with an 'inner' merge
## which keeps only the common values in both the left and right
## DFs for the result

# Checking how many values are common b/w 2 DFs
user_usage['use_id'].isin(user_device['use_id']).value_counts()

True     159
False     81
Name: use_id, dtype: int64

![Merge types](https://shanelynnwebsite-mid9n9g1q9y8tt.netdna-ssl.com/wp-content/uploads/2017/03/join-types-merge-names.jpg)

In [60]:
## Left merge

## Result will have same no. of rows as in left DF (user_usage: 240)
## with missing values for all but 159 of the merged 'platform' and
## 'device' columns
## In merge operations where a single row in the left DF is matched by
## multiple rows in the right DF, multiple result rows will be generated

result = pd.merge(user_usage,
                  user_device[['use_id', 'platform', 'device']],
                  on = 'use_id', how = 'left')

print("user_usage dimensions: {}".format(user_usage.shape))
print("result dimensions: {}".format(result.shape))
print("There are {} missing values in the result.".format(
  result['device'].isnull().sum()))

user_usage dimensions: (240, 4)
result dimensions: (240, 6)
There are 81 missing values in the result.


In [61]:
result.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


In [62]:
result.tail()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
235,260.66,68.44,896.96,25008,NaN,NaN
236,97.12,36.50,2815.00,25040,NaN,NaN
237,355.93,12.37,6828.09,25046,NaN,NaN
238,632.06,120.46,1453.16,25058,NaN,NaN
239,488.70,906.92,3089.85,25220,NaN,NaN


In [63]:
## Right Join

## Result will have same no. of rows as the right DF (user_device)
## but have several empty (NaN) values in the columns originating in
## the left DF (user_usage), namely 'outgoing_mins_per_month',
## 'outgoing_sms_per_month' and 'monthly_mb'.
## Conversely, we expect no missing values in the columns originating
## in the right DF (user_device)

result = pd.merge(user_usage,
                  user_device[['use_id', 'platform', 'device']],
                  on = 'use_id', how = 'right')

print('user_device dimensions: {}'.format(user_device.shape))
print('result dimensions: {}'.format(result.shape))
print("There are {} missing values in the 'monthly_mb' column in the result.".format(
  result['monthly_mb'].isnull().sum()))
print("There are {} missing values in the 'platform' column in the result.".format(
  result['platform'].isnull().sum()))

user_device dimensions: (272, 6)
result dimensions: (272, 6)
There are 113 missing values in the 'monthly_mb' column in the result.
There are 0 missing values in the 'platform' column in the result.


In [64]:
## Outer / Full merge

## It's a combination of left and right joins. Here, every row from
## the left and right DFs is retained in the result, with NaNs where
## there are no matched join variables.
## The result will have the same no. of rows as there are distinct
## values of 'use_id' b/w user_device and user_usage i.e. every
## join value from the left DF will be in the result along with
## every value from the right DF, and they'll be linked where possible.

print("There are {} unique values of use_id in our dataframes".format(
  pd.concat([user_usage['use_id'], user_device['use_id']]).unique().shape[0]))

result = pd.merge(user_usage,
                  user_device[['use_id', 'platform', 'device']],
                  on = 'use_id', how = 'outer')

print("Outer merge result has {} rows.".format(result.shape[0]))

print("There are {} rows with no missing values.".format(
  (result.apply(lambda x: x.isnull().sum(), axis = 1) == 0).sum()))

There are 353 unique values of use_id in our dataframes
Outer merge result has 353 rows.
There are 159 rows with no missing values.


In [0]:
## Using merge indicator to track merges

## 'indicator' parameter assists with the identification of where
## rows originate from. The '_merge' column in the output labels the
## original source for each row

result = pd.merge(user_usage,
                  user_device[['use_id', 'platform', 'device']],
                  on = 'use_id',
                  how = 'outer',
                  indicator = True)

In [66]:
result.iloc[[0, 1, 200, 201, 350, 351]]

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device,_merge
0,21.97,4.82,1557.33,22787,android,GT-I9505,both
1,1710.08,136.88,7267.55,22788,android,SM-G930F,both
200,28.79,29.42,3114.67,23988,NaN,NaN,left_only
201,616.56,99.85,5414.14,24006,NaN,NaN,left_only
350,NaN,NaN,NaN,23050,ios,"iPhone7,2",right_only
351,NaN,NaN,NaN,23051,ios,"iPhone7,2",right_only


![alt text](https://shanelynnwebsite-mid9n9g1q9y8tt.netdna-ssl.com/wp-content/uploads/2017/03/pandas-merge-outer-join-example-1-1024x357.png)

In [67]:
## Final merge - joining device details to result

## Using the inner merge, we only have entries for users where there
## is also device info. We'll redo this merge using a left join to
## keep all users, and then use a second left merge to finally get
## the device manufacturers in the same DF.

# 1st, add the platform and device to the user_usage - use a left 
# join this time

result = pd.merge(user_usage,
                  user_device[['use_id', 'platform', 'device']],
                  on = 'use_id',
                  how = 'left')

# At this point, the platform and device columns are included in the
# result along with all columns from user_usage

# Now, based on the 'device' column in result, match the 'Model'
# column in android_devices.

android_devices.rename(columns = {"Retail Branding": "manufacturer"},
                       inplace = True)

result = pd.merge(result,
                  android_devices[['manufacturer', 'Model']],
                  left_on = 'device',
                  right_on = 'Model',
                  how = 'left')

result.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device,manufacturer,Model
0,21.97,4.82,1557.33,22787,android,GT-I9505,Samsung,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F,Samsung,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F,Samsung,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303,Sony,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F,Samsung,SM-G361F


In [0]:
## Using left_on and right_on to merge with different column names

## Different column names are specified for merges in Pandas using
## the 'left_on' and 'right_on' parameters, instead of using only 
## the 'on' parameter

![alt text](https://shanelynnwebsite-mid9n9g1q9y8tt.netdna-ssl.com/wp-content/uploads/2017/03/pandas-merge-join-different-variable-names-copy-e1488722312527.png)

In [69]:
## Calculating statistics based on device

## Let's calculate the mean usage for users based on device
## manufacturer.

result.groupby("manufacturer").agg({
    "outgoing_mins_per_month": "mean",
    "outgoing_sms_per_month": "mean",
    "monthly_mb": "mean",
    "use_id": "count"
})

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
manufacturer,,,,
HTC,299.842955,93.059318,5144.077955,44
Huawei,81.526667,9.500000,1561.226667,3
LGE,111.530000,12.760000,1557.330000,2
Lava,60.650000,261.900000,12458.670000,2
Lenovo,215.920000,12.930000,1557.330000,2
Motorola,95.127500,65.666250,3946.500000,16
OnePlus,354.855000,48.330000,6575.410000,6
Samsung,191.010093,92.390463,4017.318889,108
Sony,177.315625,40.176250,3212.000625,16


## Merging on Index

In [70]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})

left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [71]:
right1 = pd.DataFrame({'group_val': [3.5, 7]},
                      index = ['a', 'b'])

right1

,group_val
a,3.5
b,7.0


In [72]:
pd.merge(left1, right1, left_on = 'key', right_index = True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [73]:
pd.merge(left1, right1, left_on = 'key', right_index = True,
         how = 'outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [74]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})

lefth

,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


In [75]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index = [['Nevada', 'Nevada', 'Ohio', 'Ohio',
                                'Ohio', 'Ohio'],
                               [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns = ['event1', 'event2'])

righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [76]:
pd.merge(lefth, righth, left_on = ['key1', 'key2'],
         right_index = True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


In [77]:
pd.merge(lefth, righth, left_on = ['key1', 'key2'],
         right_index = True, how = 'outer')

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Nevada,2001,0.0,1.0
4,4.0,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2.0,3.0


In [78]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index = ['a', 'c', 'e'],
                     columns = ['Ohio', 'Nevada'])

left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [79]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index = ['b', 'c', 'd', 'e'],
                      columns = ['Missouri', 'Alabama'])

right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [80]:
pd.merge(left2, right2, how = 'outer', left_index = True,
         right_index = True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [81]:
left2.join(right2, how = 'outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [82]:
left1.join(right1, on = 'key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [83]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index = ['a', 'c', 'e', 'f'],
                       columns = ['New York', 'Oregon'])

another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [84]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [85]:
left2.join([right2, another], how = 'outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


## Concatenating Along an Axis

In [86]:
arr = np.arange(12).reshape((3, 4))

arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [87]:
np.concatenate([arr, arr], axis = 1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [88]:
s1 = pd.Series([0, 1], index = ['a', 'b'])

s1

a    0
b    1
dtype: int64

In [89]:
s2 = pd.Series([2, 3, 4], index = ['c', 'd', 'e'])

s2

c    2
d    3
e    4
dtype: int64

In [90]:
s3 = pd.Series([5, 6], index = ['f', 'g'])

s3

f    5
g    6
dtype: int64

In [91]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [92]:
pd.concat([s1, s2, s3], axis = 1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [93]:
s4 = pd.concat([s1, s3])

s4

a    0
b    1
f    5
g    6
dtype: int64

In [94]:
pd.concat([s1, s4], axis = 1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [95]:
pd.concat([s1, s4], axis = 1, join = 'inner')

,0,1
a,0,0
b,1,1


In [96]:
pd.concat([s1, s4], axis = 1, join_axes = [['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


In [97]:
result = pd.concat([s1, s1, s3], keys = ['one', 'two', 'three'])

result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [98]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [99]:
pd.concat([s1, s2, s3], axis = 1, keys = ['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [100]:
df1 = pd.DataFrame(np.arange(6.).reshape(3, 2),
                   index = ['a', 'b', 'c'],
                   columns = ['one', 'two']) 

df1

,one,two
a,0.0,1.0
b,2.0,3.0
c,4.0,5.0


In [101]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), 
                   index = ['a', 'c'],
                   columns = ['three', 'four'])

df2

,three,four
a,5,6
c,7,8


In [102]:
pd.concat([df1, df2], axis = 1, keys = ['level1', 'level2'])

level1      level2     
     one  two  three four
a    0.0  1.0    5.0  6.0
b    2.0  3.0    NaN  NaN
c    4.0  5.0    7.0  8.0

In [103]:
pd.concat({'level1': df1, 'level2': df2}, axis = 1)

level1      level2     
     one  two  three four
a    0.0  1.0    5.0  6.0
b    2.0  3.0    NaN  NaN
c    4.0  5.0    7.0  8.0

In [104]:
pd.concat([df1, df2], axis = 1, keys = ['level1', 'level2'],
          names = ['upper', 'lower'])

upper level1      level2     
lower    one  two  three four
a        0.0  1.0    5.0  6.0
b        2.0  3.0    NaN  NaN
c        4.0  5.0    7.0  8.0

In [105]:
df1 = pd.DataFrame(np.random.randn(3, 4), 
                   columns = ['a', 'b', 'c', 'd'])

df1

,a,b,c,d
0,-1.359539,-0.277068,0.810522,0.078080
1,-0.831829,0.390258,1.040386,-0.777013
2,-0.623663,-0.829646,1.937744,1.410517


In [106]:
df2 = pd.DataFrame(np.random.randn(2, 3),
                   columns = ['b', 'd', 'a'])

df2

,b,d,a
0,-0.865963,-0.907688,0.750469
1,-0.727785,-1.196002,0.766341


In [107]:
pd.concat([df1, df2], ignore_index = True)

,a,b,c,d
0,-1.359539,-0.277068,0.810522,0.078080
1,-0.831829,0.390258,1.040386,-0.777013
2,-0.623663,-0.829646,1.937744,1.410517
3,0.750469,-0.865963,NaN,-0.907688
4,0.766341,-0.727785,NaN,-1.196002


In [108]:
# Prevent the result from including duplicate index values with
# the verify_integrity option

df5 = pd.DataFrame([1], index = ['a'])

df5

,0
a,1


In [109]:
df6 = pd.DataFrame([2], index = ['a'])

df6

,0
a,2


In [0]:
# pd.concat([df5, df6], verify_integrity = True)

# gives ValueError: Indexes have overlapping values

## Combining Data with Overlap

In [111]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index = ['f', 'e', 'd', 'c', 'b', 'a'])

a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [112]:
b = pd.Series(np.arange(len(a), dtype = np.float64),
              index = ['f', 'e', 'd', 'c', 'b', 'a'])

b[-1] = np.nan

b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [113]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [114]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [115]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})

df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [116]:
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [117]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# 8.3 Reshaping and Pivoting

## Reshaping with Hierarchical Indexing

In [118]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index = pd.Index(['Ohio', 'Colorado'], name = 'state'),
                    columns = pd.Index(['one', 'two','three'],
                                       name = 'number'))

data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [119]:
result = data.stack()

result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [120]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [121]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [122]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [0]:
s1 = pd.Series([0, 1, 2, 3], index = ['a', 'b', 'c', 'd'])

s2 = pd.Series([4, 5, 6], index = ['c', 'd', 'e'])

In [124]:
data2 = pd.concat([s1, s2], keys = ['one', 'two'])

data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [125]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [126]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [127]:
data2.unstack().stack(dropna = False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [128]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns = pd.Index(['left', 'right'], name = 'side'))

df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [129]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [130]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

## Pivoting "Long" to "Wide" Format

In [131]:
uploaded = files.upload()

Saving macrodata.csv to macrodata.csv


In [0]:
data = pd.read_csv("macrodata.csv")

In [133]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [0]:
periods = pd.PeriodIndex(year = data.year, quarter = data.quarter,
                         name = 'date')

In [0]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name = 'item')

In [0]:
data = data.reindex(columns = columns)

In [0]:
data.index = periods.to_timestamp('D', 'end')

In [0]:
ldata = data.stack().reset_index().rename(columns = {0: 'value'})

In [139]:
ldata[:10]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204


In [140]:
pivoted = ldata.pivot('date', 'item', 'value')

pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2
1960-06-30,0.14,2834.390,5.2
1960-09-30,2.70,2839.022,5.6
1960-12-31,1.21,2802.616,6.3
1961-03-31,-0.40,2819.264,6.8


In [141]:
ldata['value2'] = np.random.randn(len(ldata))

ldata[:10]

,date,item,value,value2
0,1959-03-31,realgdp,2710.349,-0.828207
1,1959-03-31,infl,0.000,-0.607638
2,1959-03-31,unemp,5.800,-1.168635
3,1959-06-30,realgdp,2778.801,-0.939756
4,1959-06-30,infl,2.340,-0.359905
5,1959-06-30,unemp,5.100,0.814014
6,1959-09-30,realgdp,2775.488,1.641701
7,1959-09-30,infl,2.740,-1.597449
8,1959-09-30,unemp,5.300,0.677676
9,1959-12-31,realgdp,2785.204,-0.484881


In [142]:
pivoted = ldata.pivot('date', 'item')

pivoted[:5]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8 -0.607638 -0.828207 -1.168635
1959-06-30  2.34  2778.801   5.1 -0.359905 -0.939756  0.814014
1959-09-30  2.74  2775.488   5.3 -1.597449  1.641701  0.677676
1959-12-31  0.27  2785.204   5.6 -1.622111 -0.484881 -1.676571
1960-03-31  2.31  2847.699   5.2 -0.382444 -1.477902  0.020678

In [143]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


In [144]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')

unstacked[:7]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8 -0.607638 -0.828207 -1.168635
1959-06-30  2.34  2778.801   5.1 -0.359905 -0.939756  0.814014
1959-09-30  2.74  2775.488   5.3 -1.597449  1.641701  0.677676
1959-12-31  0.27  2785.204   5.6 -1.622111 -0.484881 -1.676571
1960-03-31  2.31  2847.699   5.2 -0.382444 -1.477902  0.020678
1960-06-30  0.14  2834.390   5.2  1.531075 -1.159862 -0.745131
1960-09-30  2.70  2839.022   5.6  0.071216  0.313143  0.991701

## Pivoting "Wide" to "Long" Format

In [145]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})

df

,A,B,C,key
0,1,4,7,foo
1,2,5,8,bar
2,3,6,9,baz


In [146]:
melted = pd.melt(df, ['key'])

melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [147]:
reshaped = melted.pivot('key', 'variable', 'value')

reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [148]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [149]:
pd.melt(df, id_vars= ['key'], value_vars = ['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [150]:
pd.melt(df, value_vars = ['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [151]:
pd.melt(df, value_vars = ['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
